In [2]:
## imports 
import pandas as pd
import numpy as np
import yaml
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# comment these out if you don't have plotnine--not essential here/only used once
import matplotlib.pyplot as plt
# import plotnine
# from plotnine import *

## way to connect to mysql 
## if you need to install
## uncomment this line:
#! pip install mysql-connector-python
import mysql.connector

## function to feed path name to load
## credentials
def load_creds(path: str):
    with open(path, 'r') as stream:
        try:
            creds = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return(creds)

pd.options.display.max_rows = 999
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Preliminary: define connection and read sample of data

In [3]:
## read in creds; change the path name if stored
## elsewhere
creds = load_creds("C:/Users/ichir/Documents/GitHub/QSS20_S24_Sankey/public_data/09_db_cred.yaml")

In [4]:
creds

{'practice_database': {'user': 'qss20',
  'password': 'xtPdDZHnxKMFSD5RyP',
  'port': 3306,
  'database': 'sentencing',
  'host': 'qss20-7570-db.c.dartmouth.edu'}}

# Activity 1

1. Create a new column -- `in_chicago` when pulling from the `caseinit` table that takes on the value of "YES" if INCIDENT_CITY = Chicago; "NO" otherwise (which represents incidents in Cook County suburbs outside the city limits);  and pull the table. Use `crosstabs` to confirm that this worked
2. Repeat step 1 but also filter out blank strings (`INCIDENT_CITY` == "")
3. Use `where` to row filter to initiations in Chicago and use group by to find the count of cases diverted and not diverted (`is_in_diversion`); pull the table with those counts
4. Modify the query in step 3 to find the proportion of cases in chicago diverted (hint you made need to use case when in a subquery)
5. Modify the query in step 4 to find the proportion of cases in chicago versus cases not in chicago sent to diversion 


In [21]:
# your code here 1
cnx = mysql.connector.connect(user=creds['practice_database']['user'], 
                            password=creds['practice_database']['password'],
                            port=creds['practice_database']['port'],
                            database= creds['practice_database']['database'],
                            host = creds['practice_database']['host'])
cnx

In [23]:
# your code here 2

# Step 1: Create a new column 'in_chicago'
query_step_1 = """
SELECT *,
       CASE 
           WHEN INCIDENT_CITY = 'Chicago' THEN 'YES'
           ELSE 'NO'
       END AS in_chicago
FROM caseinit
"""

# Execute the query
read_step_1 = pd.read_sql_query(query_step_1, cnx)
print("Step 1: New column 'in_chicago'")
print(read_step_1.head())

# Step 2: Repeat step 1 but also filter out blank strings (INCIDENT_CITY == "")
query_step_2 = """
SELECT *,
       CASE 
           WHEN INCIDENT_CITY = 'Chicago' THEN 'YES'
           ELSE 'NO'
       END AS in_chicago
FROM caseinit
WHERE INCIDENT_CITY <> ''
"""

# Execute the query
read_step_2 = pd.read_sql_query(query_step_2, cnx)
print("\nStep 2: New column 'in_chicago' with blank strings filtered out")
print(read_step_2.head())

# Step 3: Use WHERE to row filter to initiations in Chicago and use GROUP BY to find the count of cases diverted and not diverted (is_in_diversion)
query_step_3 = """
SELECT in_chicago,
       SUM(CASE WHEN is_in_diversion = 'Yes' THEN 1 ELSE 0 END) AS cases_diverted,
       SUM(CASE WHEN is_in_diversion = 'No' THEN 1 ELSE 0 END) AS cases_not_diverted
FROM caseinit
WHERE INCIDENT_CITY = 'Chicago'
GROUP BY in_chicago
"""

# Execute the query
read_step_3 = pd.read_sql_query(query_step_3, cnx)
print("\nStep 3: Count of cases diverted and not diverted in Chicago")
print(read_step_3)

# Step 4: Modify the query in step 3 to find the proportion of cases in Chicago diverted
query_step_4 = """
SELECT in_chicago,
       SUM(CASE WHEN is_in_diversion = 'Yes' THEN 1 ELSE 0 END) / COUNT(*) AS proportion_diverted
FROM caseinit
WHERE INCIDENT_CITY = 'Chicago'
GROUP BY in_chicago
"""

# Execute the query
read_step_4 = pd.read_sql_query(query_step_4, cnx)
print("\nStep 4: Proportion of cases in Chicago diverted")
print(read_step_4)

# Step 5: Modify the query in step 4 to find the proportion of cases in Chicago versus cases not in Chicago sent to diversion
query_step_5 = """
SELECT in_chicago,
       SUM(CASE WHEN is_in_diversion = 'Yes' THEN 1 ELSE 0 END) / COUNT(*) AS proportion_diverted,
       (SELECT SUM(CASE WHEN is_in_diversion = 'Yes' THEN 1 ELSE 0 END) / COUNT(*)
        FROM caseinit
        WHERE INCIDENT_CITY != 'Chicago') AS proportion_diverted_not_in_chicago
FROM caseinit
WHERE INCIDENT_CITY = 'Chicago'
GROUP BY in_chicago
"""

# Execute the query
read_step_5 = pd.read_sql_query(query_step_5, cnx)
print("\nStep 5: Proportion of cases in Chicago versus cases not in Chicago sent to diversion")
print(read_step_5)


Step 1: New column 'in_chicago'
        CASE_ID  CASE_PARTICIPANT_ID           RECEIVED_DATE  \
0  198055620664          85937621020  08/15/1984 12:00:00 AM   
1  198055620664          85937621020  08/15/1984 12:00:00 AM   
2  198055620664          85937621020  08/15/1984 12:00:00 AM   
3  198055620664          85937621020  08/15/1984 12:00:00 AM   
4  198055620664          85937621020  08/15/1984 12:00:00 AM   

    OFFENSE_CATEGORY PRIMARY_CHARGE_FLAG      CHARGE_ID  CHARGE_VERSION_ID  \
0  PROMIS Conversion               False  1242195814523       155656315869   
1  PROMIS Conversion               False  1242198287388       131513547452   
2  PROMIS Conversion               False  1242351605056       176626576281   
3  PROMIS Conversion               False  1242352841488       176617824190   
4  PROMIS Conversion               False  1242356550787       131238606761   

  DISPOSITION_CHARGED_OFFENSE_TITLE  CHARGE_COUNT        DISPOSITION_DATE  \
0               FIRST DEGREE MURDER  

In [ ]:
# your code here 3

In [ ]:
# your code here 4

In [ ]:
# your code here 5

# Activity 2 

1. Use the following crosswalk and the `CASE` variable in the `divert` table to create a new variable `DIVERSION_PROGRAM_TEXT` that spells out the diversion programs
    - DC: Drug Court

    - DDPP: Drug Deferred Prosecution

    - DS: Drug School

    - RJCC: Restorative Justice

    - MHC: Mental Health Court

    - VC: Veteran Court

2. Build on the query from step 1 to filter to Narcotics as the `UPDATED_OFFENSE_CATEGORY` and Black or White defendants (based on race in the diversions table) (hint: you'll need to join with the caseinit table based on case_id and case_participant_id, you can do a inner join to keep only those diverted). Select the case_id, case_participant_id, case, race, and diversion_program_text columns

In [ ]:
# your code here 1

In [ ]:
# your code here 2